In [1]:

!pip install osm --no-binary :all:

/var/folders/ys/jxxv1yt57zlfqd9gmyxrs_nh0000gn/T/ipykernel_76300/2955220175.py:16: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


'1.7.1'

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
from math import exp as e
import matplotlib.pyplot as plt
import time
import networkx as nx
import osmnx as ox
from pandana.loaders import osm
from shapely.geometry import Point, Polygon
import multiprocessing as mp
import io
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
%matplotlib inline
ox.config(use_cache=True, log_console=True)
ox.__version__

DriverError: C:\Users\guret\Desktop\UrbanLogistics_Summer\Accesibilidad\celdas_lineas_rm_549_2019.shp: No such file or directory

In [ ]:
#bd_celdas_rm = (r'D:\TESIS PHD ING TRANSPORTES\Investigación 1\Definitivo\Predios_Lineas\Lineas_Log_Macrozona\Lineas RM\celdas_lineas_rm_549_2015_COM.shp')
bd_celdas_rm = (r'C:\Users\guret\Desktop\UrbanLogistics_Summer\Accesibilidad\celdas_lineas_rm_549_2019.shp')
celdas_rm = gpd.read_file(bd_celdas_rm)

In [ ]:
celdas_rm['centroid'] = celdas_rm.centroid

In [ ]:
display(celdas_rm)

In [ ]:
celdas_rm["lat"] = celdas_rm.centroid.x
celdas_rm["lon"] = celdas_rm.centroid.y

In [ ]:
display(celdas_rm)

In [ ]:
celdas_rm.columns

In [ ]:
# Definir nuestras funciones de accesibilidad (caminata y vehiculos)

## Vehiculos (Exp Negativa)
power = -0.00072779540527115486884128506462084  #ln0,5/952,39290539585 #ln0,5/46,52
f_acc_veh = lambda d: e(1)**(power*d)

def plot(f, start = 1, end = 10000, n = 200): #minutos
    xs = np.linspace(start, end, n)
    fig = plt.figure()
    plt.plot(xs, [f(x) for x in xs], 'r')
    plt.show()

## Bicicletas (Exp Negativa)
#power = -0.09902102579427790134531887449403 #ln0,25/14
#f_acc_bici = lambda d: e(1)**(power*d)

plot(f_acc_veh)
#plot(f_acc_bici)

In [ ]:
## Funciones de distancia y tiempo
def distancia(celdas_rm, id_origen, id_destino):
    ori = celdas_rm.loc[celdas_rm['cell'] == id_origen]['centroid'].iloc[0]
    des = celdas_rm.loc[celdas_rm['cell'] == id_destino]['centroid'].iloc[0]
    d = ori.distance(des)
    return d

def tiempo(celdas_rm, modo, id_origen, id_destino):
    dist_m = distancia(celdas_rm, id_origen, id_destino)
    if modo == 'veh':
        v = 25 # km/h
    elif modo == 'bici':
        v = 14 #km/h
    t = ((dist_m/1000)/v)*60
    return t

In [ ]:
#def acc_lc399(celdas_rm, modo, celda):
#    if modo == 'veh':
#        celdas_rm['aux'] = celdas_rm.apply(lambda row: row['M2_Com9_Li']*f_acc_veh(distancia(celdas_rm, celda, row['cell'])), axis = 1)
#    elif modo == 'bici':
#        celdas_rm['aux'] = celdas_rm.apply(lambda row: row['M2_Com9_Li']*f_acc_bici(distancia(celdas_rm, celda, row['cell'])), axis = 1)
#    acc_total = celdas_rm['aux'].sum()
#    return acc_total

def acc_com399(celdas_rm, modo, celda):
    if modo == 'veh':
        celdas_rm['aux'] = celdas_rm.apply(lambda row: row['M2_Com399']*f_acc_veh(distancia(celdas_rm, celda, row['cell'])), axis = 1)
    elif modo == 'bici':
        celdas_rm['aux'] = celdas_rm.apply(lambda row: row['M2_Com399']*f_acc_bici(distancia(celdas_rm, celda, row['cell'])), axis = 1)
    acc_total = celdas_rm['aux'].sum()
    return acc_total

def acc_com400(celdas_rm, modo, celda):
    if modo == 'veh':
        celdas_rm['aux'] = celdas_rm.apply(lambda row: row['M2_Com400']*f_acc_veh(distancia(celdas_rm, celda, row['cell'])), axis = 1)
    elif modo == 'bici':
        celdas_rm['aux'] = celdas_rm.apply(lambda row: row['M2_Com400']*f_acc_bici(distancia(celdas_rm, celda, row['cell'])), axis = 1)
    acc_total = celdas_rm['aux'].sum()
    return acc_total

def acc_hab(celdas_rm, modo, celda):
    if modo == 'veh':
        celdas_rm['aux'] = celdas_rm.apply(lambda row: row['M2_HAB']*f_acc_veh(distancia(celdas_rm, celda, row['cell'])), axis = 1)
    elif modo == 'bici':
        celdas_rm['aux'] = celdas_rm.apply(lambda row: row['M2_HAB']*f_acc_bici(distancia(celdas_rm, celda, row['cell'])), axis = 1)
    acc_total = celdas_rm['aux'].sum()
    return acc_total

def acc_est(celdas_rm, modo, celda):
    if modo == 'veh':
        celdas_rm['aux'] = celdas_rm.apply(lambda row: row['M2_EST']*f_acc_veh(distancia(celdas_rm, celda, row['cell'])), axis = 1)
    elif modo == 'bici':
        celdas_rm['aux'] = celdas_rm.apply(lambda row: row['M2_EST']*f_acc_bici(distancia(celdas_rm, celda, row['cell'])), axis = 1)
    acc_total = celdas_rm['aux'].sum()
    return acc_total

In [ ]:
#celdas_rm['acc_lc399'] = celdas_rm.apply(lambda row: acc_lc399(celdas_rm, 'veh', row['cell']), axis = 1)

In [ ]:
%%time
celdas_rm['acc_com399'] = celdas_rm.apply(lambda row: acc_com399(celdas_rm, 'veh', row['cell']), axis = 1)

In [ ]:
%%time
celdas_rm['acc_com400'] = celdas_rm.apply(lambda row: acc_com400(celdas_rm, 'veh', row['cell']), axis = 1)

In [ ]:
%%time
celdas_rm['acc_hab'] = celdas_rm.apply(lambda row: acc_hab(celdas_rm, 'veh', row['cell']), axis = 1)

In [ ]:
%%time
celdas_rm['acc_est'] = celdas_rm.apply(lambda row: acc_est(celdas_rm, 'veh', row['cell']), axis = 1)

In [ ]:
display(celdas_rm)

In [ ]:
celdas_rm = celdas_rm.drop(["centroid"], axis = 1)

In [ ]:
celdas_rm.to_file("acc_2019.shp")